<a href="https://colab.research.google.com/github/zhpinkman/naive-bayes-classifier/blob/master/comments-sentiment-classification-naive-bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Install PyDrive
!pip install PyDrive

#Import modules
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1uoE9erUa6gfqVpCZ9Ejoetg-RCgLYSjM"})   
downloaded.GetContentFile('comment_train.csv')       

downloaded = drive.CreateFile({'id':"1DZ5b54KJqOlioaXH8N9H37l4ldg6bLGp"})   
downloaded.GetContentFile('comment_validation.csv')       

train_df = pd.read_csv('comment_train.csv')
val_df = pd.read_csv('comment_validation.csv')

In [ ]:
train_df.head()

,title,comment,recommend
0,زیبا اما کم دوام,با وجود سابقه خوبی که از برند ایرانی نهرین سرا...,not_recommended
1,بسیار عالی,بسیار عالی,recommended
2,سلام,من الان ۳ هفته هست استفاده میکنم\r\nبرای کسایی...,not_recommended
3,به درد نمیخورهههه,عمرش کمه تا یه هفته بیشتر نمیشه استفاده کرد یا...,not_recommended
4,کلمن آب,فکر کنین کلمن بخرین با ذوق. کلی پولشو بدین. به...,not_recommended


In [ ]:
def numeric_transform(item):
    if item == 'recommended':
        return 1
    else:
        return 0
train_df['recommend'] = train_df['recommend'].apply(numeric_transform)

In [ ]:
train_df.head()

,title,comment,recommend
0,زیبا اما کم دوام,با وجود سابقه خوبی که از برند ایرانی نهرین سرا...,0
1,بسیار عالی,بسیار عالی,1
2,سلام,من الان ۳ هفته هست استفاده میکنم\r\nبرای کسایی...,0
3,به درد نمیخورهههه,عمرش کمه تا یه هفته بیشتر نمیشه استفاده کرد یا...,0
4,کلمن آب,فکر کنین کلمن بخرین با ذوق. کلی پولشو بدین. به...,0


In [ ]:
def get_propotions(df):
    positive_proportion, negative_proportion = df['recommend'].value_counts().tolist()
    positive_proportion /= len(df)
    negative_proportion /= len(df)
    return positive_proportion, negative_proportion    

In [ ]:
positive_proportion, negative_proportion = get_propotions(train_df)
positive_comments = positive_proportion * len(train_df)
negative_comments = negative_proportion * len(train_df)
print('proportion of the positive comments: ', positive_proportion)
print('proportion of the nagative comments: ', negative_proportion)
print('number of positive comments:', positive_comments)
print('number of positive comments:', negative_comments)

proportion of the positive comments:  0.5
proportion of the nagative comments:  0.5
number of positive comments: 3000.0
number of positive comments: 3000.0


In [ ]:
def remove_newline(df):
    df.comment = df.comment.replace(r'\n',  ' ', regex=True)
    df.comment = df.comment.replace(r'\r',  ' ', regex=True)
    df.comment = df.comment.replace(r'\.',  ' ', regex=True)    
    return df

In [ ]:
train_df = remove_newline(train_df)
train_df.head().comment.values.tolist()

['با وجود سابقه خوبی که از برند ایرانی نهرین سراغ داشتم اما متاسفانه بعد از فقط 15 روز رویه کفش ترک خورده و کناره ها هم شروع کرده به جدا شدن از کفه، فقط در فروش ویژه ممکنه به صرفه باشه ',
 'بسیار عالی',
 'من الان ۳ هفته هست استفاده میکنم  برای کسایی که در طول ماه دو سه تا دونه جوش میزنن خیلی خوبه که نقطه نقطه روی اون جوش ها بزنید و بعد از ۲ روز خشک\u200cمیشه  اما من صورتم خیلی جوش داره و جوش زیر پوستی هم داده  برای من مناسب نبود چون با چند بار استفاده قشنگ صورتم سووووخت شروع به ورقه ورقه شدن کرد و خوب ظاهر پوستم خیلی بد شد مخصوصا زیر آرایش کامل مشششخصه تیکه تیکه شده پوستم  ، حدود ۲ هفته طول کشید درست شد و پوست جدید جایگزین شد  الان یه شب در میون میزنم ولی همچنان جوش زیر پوستی دارم بازم جوش فعال میزنم در صورتی که در توضیحاتش نوشته بعد از یک هفته استفاده اصلااا نباید دیگه جوش بزنیم   پوست من چرب بود و الان مثل چوووب خشک شده که پوست میندازم هی  به کسایی که خیلی جوش دارن توصیه نمیکنم فقط به درد کسایی میخوره که ماهی دو سه تا جوووش کووچووولووو میزنن  و در ضمن فرورفتگی ناشی از جای جوش هم\u200

In [ ]:
train_df.head()

,title,comment,recommend
0,زیبا اما کم دوام,با وجود سابقه خوبی که از برند ایرانی نهرین سرا...,0
1,بسیار عالی,بسیار عالی,1
2,سلام,من الان ۳ هفته هست استفاده میکنم برای کسایی ک...,0
3,به درد نمیخورهههه,عمرش کمه تا یه هفته بیشتر نمیشه استفاده کرد یا...,0
4,کلمن آب,فکر کنین کلمن بخرین با ذوق کلی پولشو بدین به...,0


In [ ]:
def get_dict(comments):
    tokens_list = list()
    tok2total = dict()
    for comment in comments:
        tokens_list.append(comment.split())
    tokens = [token for comment_tokens in tokens_list for token in comment_tokens]
    for token in tokens:
        if not token in tok2total:
            tok2total[token] = 0
        tok2total[token] += 1 
    return tok2total

def get_total_dicts(df):
    pos_comments = df[df.recommend == 1].comment
    neg_comments = df[df.recommend == 0].comment
    pos_tok2total = get_dict(pos_comments)
    neg_tok2total = get_dict(neg_comments)
    return pos_tok2total, neg_tok2total

In [ ]:
pos_tok2total, neg_tok2total = get_total_dicts(train_df)

In [ ]:
pos_tokens_counts = sum(list(pos_tok2total.values()))
neg_tokens_counts = sum(list(neg_tok2total.values()))
all_tokens_counts = pos_tokens_counts + neg_tokens_counts

In [ ]:
print('number of words in positive comments: ', pos_tokens_counts)
print('number of words in negative comments: ', neg_tokens_counts)
print('number of words in all comments: ', all_tokens_counts)

number of words in positive comments:  112044
number of words in negative comments:  99364
number of words in all comments:  211408


In [ ]:
print('number of tokens in positive comments: ', len(pos_tok2total))
print('number of tokens in negative comments: ', len(neg_tok2total))

number of tokens in positive comments:  11815
number of tokens in negative comments:  10897


In [ ]:
pos_tok2prob = dict()
neg_tok2prob = dict()
for token in pos_tok2total:
    pos_tok2prob[token] = pos_tok2total[token] / pos_tokens_counts
for token in neg_tok2total:
    neg_tok2prob[token] = neg_tok2total[token] / neg_tokens_counts

In [ ]:
pos_tok2prob['خوب'], pos_tok2prob['بد'], pos_tok2prob['عالی']

(0.004810610117453857, 0.0004819535182606833, 0.00448038270679376)

In [ ]:
neg_tok2prob['خوب'], neg_tok2prob['بد'], neg_tok2prob['عالی']

(0.003401634394750614, 0.0017913932611408557, 0.00046294432591280544)

In [ ]:
val_df['recommend'] = val_df['recommend'].apply(numeric_transform)

In [ ]:
val_df = remove_newline(val_df)

In [ ]:
val_df.head()

,title,comment,recommend
0,وری گود,تازه خریدم یه مدت کار بکنه مشخص میشه کیفیت قطعاتش,1
1,زیاد مناسب نیست رنگ پس میده یه وقتایی موقع نوشتن,با این قیمت گزینه های بهتری هم میشه گرفت روا...,0
2,پنکه گوشی,خیلی عالیه، فقط کاش از اون سمتش میشد به پاوربا...,1
3,دستگاه خیلی ضعیف,من این فیس براس چند روز یپش به دستم رسید و الا...,0
4,عالی و بیست,بنده یه هارد اکسترنال دارم که کابل فابریکش سال...,1


In [ ]:
comments = val_df.comment
comments[:5]

0    تازه خریدم یه مدت کار بکنه مشخص میشه کیفیت قطعاتش
1    با این قیمت گزینه های بهتری هم میشه گرفت   روا...
2    خیلی عالیه، فقط کاش از اون سمتش میشد به پاوربا...
3    من این فیس براس چند روز یپش به دستم رسید و الا...
4    بنده یه هارد اکسترنال دارم که کابل فابریکش سال...
Name: comment, dtype: object

In [ ]:
results = np.empty(shape=0)
results.shape

(0,)

In [ ]:
for comment in comments:
    tokens = comment.split()
    pos_value = positive_proportion
    neg_value = negative_proportion
    for token in tokens:
        if token in pos_tok2prob:
            pos_value *= pos_tok2prob[token]
        else:
            pos_value *= 0
        if token in neg_tok2prob:
            neg_value *= neg_tok2prob[token]
        else: 
            neg_value *= 0
    results = np.append(results, (pos_value >= neg_value))

In [ ]:
def calc_acc_prec_rec_f1(df, column):
    accuracy = sum(df[column].values == df.recommend.values) / df.shape[0]
    recommended_portion = df[df.recommend == 1]
    precision = sum(recommended_portion[column].values == recommended_portion.recommend.values) / sum(df[column])
    recall = sum(recommended_portion[column].values == recommended_portion.recommend.values) / len(recommended_portion)
    f1 = 2 * precision * recall / (precision + recall)
    return accuracy, precision, recall, f1

In [ ]:
val_df['raw_results'] = results

In [ ]:
val_df.head()

,title,comment,recommend,raw_results
0,وری گود,تازه خریدم یه مدت کار بکنه مشخص میشه کیفیت قطعاتش,1,0.0
1,زیاد مناسب نیست رنگ پس میده یه وقتایی موقع نوشتن,با این قیمت گزینه های بهتری هم میشه گرفت روا...,0,0.0
2,پنکه گوشی,خیلی عالیه، فقط کاش از اون سمتش میشد به پاوربا...,1,1.0
3,دستگاه خیلی ضعیف,من این فیس براس چند روز یپش به دستم رسید و الا...,0,1.0
4,عالی و بیست,بنده یه هارد اکسترنال دارم که کابل فابریکش سال...,1,1.0


In [ ]:
calc_acc_prec_rec_f1(val_df, 'raw_results')

(0.84375, 0.7733598409542743, 0.9725, 0.8615725359911407)